# 5.1 Leave-One-Study-Out Evaluation (Baseline)

**模型架构**: Baseline (Only Species, Rel-sorted, No Value/Condition)
**任务**: 对 6 个目标 Study 进行留一法交叉验证 (LOO-CV) 和零样本迁移 (Zero-shot) 测试。

In [15]:
import os
import torch
import numpy as np
import pandas as pd
import json
import joblib
import gc
import shutil
from argparse import Namespace
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from torch.utils.data import Subset
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# --- v2.0 自定义模块 ---
from MiCoGPT.utils_vCross.model_vCross import MiCoGPTConfig, MiCoGPTForSequenceClassification
from MiCoGPT.utils_vCross.collator_vCross import MiCoGPTClassificationCollator

# --- 复用工具函数 ---
from MiCoGPT.utils.finetune_v2 import SubsetWithLabels
from MiCoGPT.utils.mgm_utils import eval_and_save

import warnings
warnings.filterwarnings("ignore")

In [16]:
args = Namespace(
    # 语料库 (vCross 标准版)
    input="../data/vCross/ResMicroDB_90338_vCross.pkl",
    
    # 基础模型 (Baseline Group A)
    base_model="../models/finetuned_vCross_baseline_valLoss",
    
    # 标签编码器
    label_encoder_path="../models/finetuned_vCross_baseline_valLoss/label_encoder.joblib",
    
    # 输出根目录
    output_root="../models/LOO_vCross_baseline_valLoss",
    
    # 目标 Studies
    target_studies = [
        'PRJNA439311', 'PRJNA282010', 'PRJNA296567', 
        'PRJNA632472', 'PRJNA1108737', 'PRJNA820972'
    ],
    
    label_col="Is_Healthy",
)

os.makedirs(args.output_root, exist_ok=True)

In [17]:
print("Loading Corpus...")
with open(args.input, "rb") as f:
    import pickle
    corpus = pickle.load(f)

print("Loading Label Encoder...")
le = joblib.load(args.label_encoder_path)

# 兼容 LabelEncoder 和 OneHotEncoder
if hasattr(le, "classes_"):
    classes = le.classes_
else:
    classes = le.categories_[0]
print(f"Classes: {classes}")

# 筛选 Split_Group == 'C' 且在目标 Studies 中的样本
meta = corpus.metadata
mask = (
    (meta["Split_Group"] == "C") & 
    (meta["Project_ID"].isin(args.target_studies)) & 
    (meta[args.label_col].notna())
)
target_indices = np.where(mask)[0]
target_subset = Subset(corpus, target_indices)
print(f"Selected {len(target_subset)} samples from {len(args.target_studies)} studies.")

Loading Corpus...
Loading Label Encoder...
Classes: [False True]
Selected 1029 samples from 6 studies.


In [18]:
# 定义 Baseline Collator (屏蔽 Value/Condition)
class BaselineClassificationCollator(MiCoGPTClassificationCollator):
    def __call__(self, examples):
        batch = super().__call__(examples)
        batch["value_ids"] = None
        batch["condition_ids"] = None
        return batch

collator = BaselineClassificationCollator(corpus.tokenizer, max_length=512)

In [19]:
def get_study_subset(study_id, indices_map):
    study_mask = (meta["Project_ID"] == study_id) & mask
    study_indices = np.where(study_mask)[0]
    return Subset(corpus, study_indices)

def prepare_dataset(subset, le):
    indices = subset.indices
    labels_raw = corpus.metadata.iloc[indices][args.label_col].values
    
    if hasattr(le, "classes_"):
        labels_encoded = le.transform(labels_raw)
    else:
        # OneHotEncoder
        labels_onehot = le.transform(labels_raw.reshape(-1, 1))
        if hasattr(labels_onehot, "toarray"):
            labels_onehot = labels_onehot.toarray()
        labels_encoded = np.argmax(labels_onehot, axis=1)
        
    return SubsetWithLabels(subset, torch.tensor(labels_encoded, dtype=torch.long))

## 任务一：Zero-shot Evaluation

In [20]:
print("=== Starting Zero-shot Evaluation ===")

model = MiCoGPTForSequenceClassification.from_pretrained(
    args.base_model, 
    num_labels=len(classes),
    ignore_mismatched_sizes=True
)

trainer = Trainer(
    model=model,
    data_collator=collator,
    args=TrainingArguments(output_dir=".tmp", per_device_eval_batch_size=32, eval_accumulation_steps=None)
)

for study in args.target_studies:
    print(f"Evaluating on {study}...")
    study_subset = get_study_subset(study, None)
    if len(study_subset) == 0: continue
    test_ds = prepare_dataset(study_subset, le)
    
    predictions = trainer.predict(test_ds)
    y_score = predictions.predictions
    y_true = predictions.label_ids
    
    save_dir = f"{args.output_root}/zero_shot/{study}"
    os.makedirs(save_dir, exist_ok=True)
    eval_and_save(y_score, y_true, [str(c) for c in classes], save_dir, activation="softmax")
    print(f"Saved to {save_dir}")

del model, trainer
gc.collect()
torch.cuda.empty_cache()

=== Starting Zero-shot Evaluation ===
Evaluating on PRJNA439311...


Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.001   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.002   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.003   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.004   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997   9   7  17  69  0.7647  0.8023  0.5625  0.8023  0.4375  0.8023  0.9079   
0.998   9   7  27  59  0.6667  0.6860  0.5625  0.6860  0.4375  0.6860  0.8939   
0.999  12   4  37  49  0.5980  0.5698  0.7500  0.5698  0.2500  0.5698  0.9245   
1.000  16   0  86   0  0.1569  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.001   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.002   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.003   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.004   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997   5  14   0  18  0.6216  1.0000  0.2632  1.0000  0.7368  1.0000  0.5625   
0.998   6  13   0  18  0.6486  1.0000  0.3158  1.0000  0.6842  1.0000  0.5806   
0.999  12   7   7  11  0.6216  0.6111  0.6316  0.6111  0.3684  0.6111  0.6111   
1.000  19   0  18   0  0.5135  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP  FN  TP     Acc    Sn      Sp   TPR     FPR    Rc      Pr  \
t                                                                         
0.000   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.001   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.002   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.003   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
0.004   0  14   0  20  0.5882  1.00  0.0000  1.00  1.0000  1.00  0.5882   
...    ..  ..  ..  ..     ...   ...     ...   ...     ...   ...     ...   
0.997  11   3  15   5  0.4706  0.25  0.7857  0.25  0.2143  0.25  0.6250   
0.998  11   3  15   5  0.4706  0.25  0.7857  0.25  0.2143  0.25  0.6250   
0.999  11   3  17   3  0.4118  0.15  0.7857  0.15  0.2143  0.15  0.5000   
1.000  14   0  20   0  0.4118  0.00  1.0000  0.00  0.0000  0.00  0.0000   
1.001  14   0  20   0  0.4118  0.00  1.0000  0.00  0.0000  0.00  0.00

Evaluating biome source: False
        TN   FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                           
0.000    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ...  ...  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  132   75  161  177  0.5670  0.5237  0.6377  0.5237  0.3623  0.5237   
0.998  148   59  180  158  0.5615  0.4675  0.7150  0.4675  0.2850  0.4675   
0.999  169   38  226  112  0.5156  0.3314  0.8164  0.3314  0.1836  0.3314   
1.000  207    0  338    0  0.3798  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  207    0  338    0  0.3798  0.0000  1.

Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  76   0  76  0.5000  1.0000  0.0000  1.0000  1.0000  1.0000  0.5000   
0.001   0  76   0  76  0.5000  1.0000  0.0000  1.0000  1.0000  1.0000  0.5000   
0.002   0  76   0  76  0.5000  1.0000  0.0000  1.0000  1.0000  1.0000  0.5000   
0.003   1  75   0  76  0.5066  1.0000  0.0132  1.0000  0.9868  1.0000  0.5033   
0.004   1  75   0  76  0.5066  1.0000  0.0132  1.0000  0.9868  1.0000  0.5033   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997  63  13  62  14  0.5066  0.1842  0.8289  0.1842  0.1711  0.1842  0.5185   
0.998  67   9  66  10  0.5066  0.1316  0.8816  0.1316  0.1184  0.1316  0.5263   
0.999  73   3  69   7  0.5263  0.0921  0.9605  0.0921  0.0395  0.0921  0.7000   
1.000  76   0  76   0  0.5000  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                         
0.000   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ..  ..  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  16  38   12   93  0.6855  0.8857  0.2963  0.8857  0.7037  0.8857   
0.998  20  34   14   91  0.6981  0.8667  0.3704  0.8667  0.6296  0.8667   
0.999  21  33   22   83  0.6541  0.7905  0.3889  0.7905  0.6111  0.7905   
1.000  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.00

## 任务二：LOO Fine-tuning

In [21]:
import numpy as np
import gc
import torch
from transformers import TrainerCallback

print("=== Starting Nested LOO (Double LOO) Fine-tuning ===")

# 辅助函数：清理显存
def clean_memory():
    gc.collect()
    torch.cuda.empty_cache()

# 辅助函数：计算指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# 定义最大搜索轮数
MAX_EPOCHS = 50

# === 1. 外层循环：留一个作为最终测试集 (Test Study) ===
for test_study in args.target_studies:
    print(f"\n{'='*20}")
    print(f">>> Outer Loop: Test Target = {test_study}")
    print(f"{'='*20}")
    
    # 确定当前的训练池 (5个 studies)
    train_pool_studies = [s for s in args.target_studies if s != test_study]
    
    # 用于存储内层 5 折的 Loss 曲线
    # 结构: [ [fold1_epoch1_loss, fold1_epoch2_loss...], [fold2...], ... ]
    all_folds_val_losses = []
    
    # === 2. 内层循环：5-Fold CV 寻找最佳 Epoch ===
    print(f"\n[Inner Loop] Starting 5-fold CV on train pool to find optimal epochs...")
    
    for fold_idx, val_study in enumerate(train_pool_studies):
        print(f"  Inner Fold {fold_idx+1}/5: Val Study = {val_study}")
        
        # 2.1 准备内层数据
        # 内层训练集：训练池中除了当前验证 Study 之外的 4 个
        inner_train_studies = [s for s in train_pool_studies if s != val_study]
        
        # 构建 Dataset
        inner_train_mask = (meta["Project_ID"].isin(inner_train_studies)) & mask
        inner_train_ds = prepare_dataset(Subset(corpus, np.where(inner_train_mask)[0]), le)
        
        inner_val_subset = get_study_subset(val_study, None)
        inner_val_ds = prepare_dataset(inner_val_subset, le)
        
        # 2.2 初始化模型 (每次都要重置)
        model = MiCoGPTForSequenceClassification.from_pretrained(
            args.base_model, 
            num_labels=len(classes),
            ignore_mismatched_sizes=True
        )
        
        # 2.3 训练配置
        # 注意：内层不需要 load_best_model_at_end，我们需要完整的 loss 曲线
        inner_output_dir = f"{args.output_root}/temp_inner/{test_study}/{val_study}"
        training_args = TrainingArguments(
            output_dir=inner_output_dir,
            num_train_epochs=MAX_EPOCHS,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            learning_rate=1e-5,
            weight_decay=0.01,
            evaluation_strategy="epoch", # 每个 epoch 测一次
            save_strategy="no",          # 内层为了省空间和时间，不保存 checkpoint
            load_best_model_at_end=False,
            logging_steps=10,
            dataloader_num_workers=0,
            report_to="none"             # 避免 wandb 等干扰
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=inner_train_ds,
            eval_dataset=inner_val_ds,
            data_collator=collator,
            compute_metrics=compute_metrics
        )
        
        trainer.train()
        
        # 2.4 提取验证 Loss 曲线
        # log_history 中包含训练 loss 和 eval loss，我们需要筛选出 eval_loss
        history = trainer.state.log_history
        # 提取包含 'eval_loss' 的记录，并按 epoch 排序
        eval_logs = [log for log in history if 'eval_loss' in log]
        # 确保按 epoch 顺序（虽然通常已经是顺序的）
        eval_logs.sort(key=lambda x: x['epoch'])
        
        fold_losses = [log['eval_loss'] for log in eval_logs]
        
        # 简单校验：如果因为某些原因记录数不够，补齐或截断（通常不会发生）
        if len(fold_losses) < MAX_EPOCHS:
            print(f"    Warning: Expected {MAX_EPOCHS} logs, got {len(fold_losses)}. Padding with infinity.")
            fold_losses += [float('inf')] * (MAX_EPOCHS - len(fold_losses))
        
        all_folds_val_losses.append(fold_losses)
        
        # 清理资源
        del model, trainer, inner_train_ds, inner_val_ds
        clean_memory()
    
    # === 3. 聚合结果，选择最佳 Epoch ===
    # 将 list 转为 numpy array: shape (5, MAX_EPOCHS)
    loss_matrix = np.array(all_folds_val_losses)
    # 计算每个 epoch 的平均 loss
    avg_loss_per_epoch = np.mean(loss_matrix, axis=0)
    
    # 找到 loss 最小的索引 (epoch 从 1 开始，所以 index + 1)
    best_epoch_idx = np.argmin(avg_loss_per_epoch)
    best_epoch = int(best_epoch_idx + 1)
    min_loss = avg_loss_per_epoch[best_epoch_idx]
    
    print(f"\n[Result] Best Epoch for {test_study} is: {best_epoch} (Avg Val Loss: {min_loss:.4f})")
    
    # === 4. Refit: 用最佳 Epoch 在整个训练池上重训 ===
    print(f"[Refit] Retraining on all 5 training studies for {best_epoch} epochs...")
    
    # 4.1 准备 Refit 数据 (所有 5 个训练 study 合并)
    refit_train_mask = (meta["Project_ID"].isin(train_pool_studies)) & mask
    refit_train_ds = prepare_dataset(Subset(corpus, np.where(refit_train_mask)[0]), le)
    
    # 准备最终测试数据
    final_test_subset = get_study_subset(test_study, None)
    final_test_ds = prepare_dataset(final_test_subset, le)
    
    print(f"  Refit Train Samples: {len(refit_train_ds)}")
    print(f"  Final Test Samples:  {len(final_test_ds)}")
    
    # 4.2 初始化模型
    model = MiCoGPTForSequenceClassification.from_pretrained(
        args.base_model, 
        num_labels=len(classes),
        ignore_mismatched_sizes=True
    )
    
    # 4.3 训练配置
    final_output_dir = f"{args.output_root}/finetuned/{test_study}"
    training_args = TrainingArguments(
        output_dir=final_output_dir,
        num_train_epochs=best_epoch,     # <--- 关键：使用选出的最佳 Epoch
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        learning_rate=1e-5,
        weight_decay=0.01,
        evaluation_strategy="no",        # Refit 阶段不需要验证
        save_strategy="no",              # Refit 结束后直接保存或预测，不需要中间存 checkpoint
        logging_steps=10,
        dataloader_num_workers=0
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=refit_train_ds,
        data_collator=collator
    )
    
    trainer.train()
    
    # === 5. 最终预测与保存 ===
    print(f"[Predict] Predicting on {test_study}...")
    predictions = trainer.predict(final_test_ds)
    
    eval_and_save(
        predictions.predictions, 
        predictions.label_ids, 
        [str(c) for c in classes], 
        final_output_dir, 
        activation="softmax"
    )
    
    print(f"Saved results to {final_output_dir}")
    
    # 记录最佳参数信息到文件（可选）
    with open(f"{final_output_dir}/best_params.txt", "w") as f:
        f.write(f"Best Epoch: {best_epoch}\n")
        f.write(f"Avg Val Loss: {min_loss:.4f}\n")
        f.write(f"Loss Curve: {avg_loss_per_epoch.tolist()}\n")
    
    # 清理资源
    del model, trainer, refit_train_ds, final_test_ds
    clean_memory()

print("\n=== All LOO Rounds Completed ===")

=== Starting Nested LOO (Double LOO) Fine-tuning ===

>>> Outer Loop: Test Target = PRJNA439311

[Inner Loop] Starting 5-fold CV on train pool to find optimal epochs...
  Inner Fold 1/5: Val Study = PRJNA282010


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.651000,2.253196,0.486486,0.318428
2,1.176900,1.660267,0.513514,0.375676
3,0.990400,1.401815,0.540541,0.428256
4,0.874300,1.213786,0.567568,0.476847
5,0.940100,1.089798,0.594595,0.522016
6,1.000100,1.043209,0.594595,0.522016
7,0.803900,0.938864,0.621622,0.564234
8,0.765200,0.887050,0.675676,0.641372
9,0.710800,0.836720,0.729730,0.710844
10,0.794800,0.809814,0.729730,0.710844


  Inner Fold 2/5: Val Study = PRJNA296567


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.512100,1.931465,0.588235,0.481335
2,1.295300,1.812117,0.588235,0.481335
3,1.139300,1.759940,0.588235,0.481335
4,0.859600,1.726176,0.588235,0.481335
5,0.843100,1.696456,0.588235,0.481335
6,0.755000,1.665880,0.588235,0.481335
7,0.827600,1.645755,0.588235,0.481335
8,0.762000,1.624861,0.588235,0.481335
9,0.750700,1.598245,0.588235,0.481335
10,0.775200,1.586777,0.588235,0.481335


  Inner Fold 3/5: Val Study = PRJNA632472


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.405300,1.566562,0.493421,0.471436
2,1.071200,1.473456,0.480263,0.472013
3,0.878100,1.446740,0.500000,0.494398
4,0.791100,1.434873,0.506579,0.501726
5,0.788600,1.419335,0.519737,0.516198
6,0.719300,1.409736,0.519737,0.516198
7,0.708200,1.401988,0.513158,0.511042
8,0.762200,1.389115,0.526316,0.525000
9,0.643200,1.382261,0.519737,0.518047
10,0.596500,1.374744,0.500000,0.499220



[Result] Best Epoch for PRJNA820972 is: 43 (Avg Val Loss: 1.1299)
[Refit] Retraining on all 5 training studies for 43 epochs...
  Refit Train Samples: 870
  Final Test Samples:  159


Step,Training Loss
10,1.716600
20,1.336600
30,1.295900
40,1.023800
50,1.081300
60,0.989900
70,0.983900
80,0.909200
90,0.953400
100,0.778700


[Predict] Predicting on PRJNA820972...


Evaluating biome source: False
       TN  FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                         
0.000   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ..  ..  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  48   6   79   26  0.4654  0.2476  0.8889  0.2476  0.1111  0.2476   
0.998  49   5   83   22  0.4465  0.2095  0.9074  0.2095  0.0926  0.2095   
0.999  51   3   96    9  0.3774  0.0857  0.9444  0.0857  0.0556  0.0857   
1.000  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.00